**<파일 경로>**
- C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/COVID-19

In [1]:
filepath = input()

C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/COVID-19


In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
%matplotlib inline
# seaborn 폰트 및 스타일 설정
sns.set(font = "Malgun Gothic", rc = {"axes.unicode_minus": False}, style = 'whitegrid')

# COVID-19 데이터
- 출처: 서울시 코로나19 홈페이지
    - http://www.seoul.go.kr/coronaV/coronaStatus.do
    - 2020-07-10 10:00:00 기준 자료
- 위 홈페이지를 참고하여 서울시 구별 확진자 수를 수작업으로 입력한 뒤에 Excel 파일로 만들어주었다.

In [3]:
df_covid = pd.read_excel('{}/COVID-19_200710_10시_기준.xlsx'.format(filepath))
df_covid

,gu,patient_num,lat,lng
0,관악구,131,37.465399,126.943807
1,구로구,88,37.495486,126.858121
2,강남구,87,37.495985,127.066409
3,강서구,80,37.565762,126.822656
4,양천구,70,37.527062,126.856153
5,영등포구,68,37.520641,126.913924
6,도봉구,62,37.665861,127.031767
7,동작구,60,37.506175,126.952890
8,서초구,59,37.481755,127.005501
9,송파구,56,37.504853,127.114482


# 서울시 구별 경계선을 그릴 수 있는 json 파일 얻기
- 파일 출처: [Github e9t](https://github.com/southkorea/southkorea-maps)
    - 'skorea_municipalities_geo_simple.json' 파일 다운로드

In [4]:
geo_path = '{}/skorea_municipalities_geo_simple.json'.format(filepath)
geo_json = json.load(open(geo_path, encoding = 'utf-8'))

In [5]:
# json 파일의 형태 확인
geo_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '39020',
    'name': '서귀포시',
    'name_eng': 'Seogwipo-si',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[126.1701670531016, 33.27833920373795],
      [126.17796199822318, 33.28904450342792],
      [126.20366562455006, 33.292582069125935],
      [126.23227804627976, 33.28054651709448],
      [126.27081469983565, 33.29305651947374],
      [126.28692516789717, 33.3095121006344],
      [126.32582807936741, 33.32307849544494],
      [126.33863040850332, 33.33700002289742],
      [126.37692773779688, 33.34348978209229],
      [126.42008790072302, 33.33448285889923],
      [126.43967991919219, 33.34042205120408],
      [126.44742456956253, 33.35529267692266],
      [126.4918716225209, 33.351283204155564],
      [126.54001194569452, 33.35949354681064],
      [126.55310458705854, 33.368769331336765],
      [126.5830715764017, 33.368027779973275],
      [126.65655971322

# Folium을 통한 서울시 COVID-19 발생 현황 시각화

In [6]:
# 서울시 중심부의 위도, 경도
center = [37.541, 126.986]

# covid_map이 center에 위치하고, zoom 레벨은 11로 설정
covid_map = folium.Map(location = center, zoom_start = 11)

folium.Choropleth(geo_data = geo_json, 
                  data = df_covid,
                  columns = ('gu', 'patient_num'),
                  key_on = 'feature.properties.name',
                  fill_color = 'PuRd',
                  legend_name = '서울시 코로나 확진자 수').add_to(covid_map)

for idx in df_covid.index:
    count = df_covid.loc[idx, 'patient_num']
    gu = df_covid.loc[idx, 'gu']
    msg = '{}: {}명'.format(gu, count)
    
    # CircleMarker 설정
    size = (count / df_covid['patient_num'].sum() * 500) # 지도에 표시할 원의 넓이 설정
    folium.CircleMarker([df_covid.loc[df_covid['gu'] == gu, 'lat'],  # 각 자치구별 위도 매칭
                         df_covid.loc[df_covid['gu'] == gu, 'lng']], # 각 자치구별 경도 매칭
                        radius = size,
                        color = '#3186cc',
                        fill_color = '#3186cc').add_to(covid_map)
    
    # Marker 설정
    lat = df_covid.loc[idx, 'lat'] # 각 자치구별 위도
    lng = df_covid.loc[idx, 'lng'] # 각 자치구별 경도
    folium.Marker(location = [lat, lng],
                  popup = '{} {}명'.format(gu, count),
                  icon = folium.Icon(color = 'red', icon = 'star')).add_to(covid_map)
    
covid_map

# Folium 시각화 결과를 HTML 파일로 내보내기

In [7]:
new_filepath = 'C:/Users/Brian/Desktop/Data Science/Projects/Playdata/PPT'

In [8]:
covid_map.save('{}/서울시 COVID-19 발생 현황.html'.format(new_filepath))